In [ ]:
# default_exp seed_integration_tests

In [ ]:
#export

from pathlib import Path

def seed_workflow(dev_client: object,
                  workflow_path: str,
                  upload_path: str):
    workflow_path = Path(workflow_path)
    workflow_name = workflow_path.name

    workflow_response = dev_client.files.upload_file(filename=workflow_path,
                                                     upload_path=f"{upload_path}{workflow_name}",
                                                     conflict_action="MERGE")

    assert(workflow_response.status_code == 200)

    return workflow_response.json()

def scaffold_workflows_integration_tests(client):
    questions_workflow_path = Path(r'test_workflows/simple_questions.yxwz')

    workflow_a_path = Path(r'WorkflowA.yxmd')

    workflow_b_path = Path(r'WorkflowB.yxmd')

    aah_upload_path = r'/Workspaces/Public/'

    questions_workflow = seed_workflow(client,
                                       questions_workflow_path,
                                       aah_upload_path)

    workflow_a_json = seed_workflow(client,
                                    workflow_a_path,
                                    aah_upload_path)

    workflow_b_json = seed_workflow(client,
                                    workflow_b_path,
                                    aah_upload_path)

    workflow_a_path = workflow_a_json['path']
    workflow_a_uuid = workflow_a_json['uuid']
    workflow_a_content_id = workflow_a_json['contentId']

    workflow_b_path = workflow_b_json['path']
    workflow_b_uuid = workflow_b_json['uuid']
    workflow_b_content_id = workflow_b_json['contentId']

    questions_wf_path = questions_workflow['path']
    questions_wf_uuid = questions_workflow['uuid']
    questions_wf_content_id = questions_workflow['contentId']

def fetch_all_jobs(client):
    response = client.jobs.get_job()
    jobs_list = response.json().get('jobs')
    return jobs_list

def test_can_get_jobs_with_no_args(client):
    response = client.jobs.get_job()
    assert(response.status_code == 200)

def test_that_server_has_jobs_available(client):
    jobs_list = fetch_all_jobs(client)
    assert(jobs_list is not None)
    assert(type(jobs_list) == list)
    assert(len(jobs_list) > 0)

def test_can_get_job_by_id(client):
    jobs_list = fetch_all_jobs(client)
    test_job = jobs_list[0]
    test_job_id = test_job.get('jobId')
    response = client.jobs.get_job(job_id=test_job_id)
    assert(response.json() == test_job)

def test_get_log_functionality(client):
    jobs_list = fetch_all_jobs(client)
    test_job = jobs_list[0]
    test_job_id = test_job.get('jobId')
    response = client.jobs.get_log(job_id=test_job_id)
    assert(response.status_code == 200)
    assert(response.json().get('jobId') == test_job_id)
    assert(response.json().get('entries'))
    assert(len(response.json().get('entries')) > 0)


def test_cancel_job_functionality(client):
    import time

    workflow_a_path = Path(r'WorkflowA.yxmd')

    aah_upload_path = r'/Workspaces/Public/'

    workflow_a_json = seed_workflow(client,
                                    workflow_a_path,
                                    aah_upload_path)

    workflow_a_path = workflow_a_json['path']
    workflow_a_uuid = workflow_a_json['uuid']
    workflow_a_content_id = workflow_a_json['contentId']

    run_workflow_response = client.workflows.run_workflow(
        asset_id=workflow_a_uuid,
        schedule_name="int_test"
    )

    schedule_id = run_workflow_response.json().get('scheduleId')
    if schedule_id:
        workflow_job = client.jobs.get_job(params={
            "scheduleId": schedule_id
        })
        running_job = workflow_job.json().get('jobs')[0]
        running_job_id = running_job.get('jobId')
        running_job_status = running_job.get('status')

        print(running_job_status)
        cancelled_job = client.jobs.cancel_job(job_id=running_job_id)

        assert(cancelled_job.status_code == 204)

        cancelled_job = client.jobs.get_job(job_id=running_job_id)

        assert(cancelled_job.json().get('status') == 'canceled')

    assert(schedule_id is not None)

In [ ]:
#export

def seed_workflow_files(client):
    files = client.files
    workflow_A_name = "WorkflowA.yxmd"
    workflow_B_name = "WorkflowB.yxmd"
    workflow_A = files.upload_file(workflow_A_name,
                                   upload_path=f"/Workspaces/Public/{workflow_A_name}",
                                   conflict_action="MERGE").json()
    workflow_B = files.upload_file(workflow_B_name,
                                   upload_path=f"/Workspaces/Public/{workflow_B_name}",
                                   conflict_action="MERGE").json()
    return (workflow_A, workflow_B)